In [61]:
#Used Faker and Random Library to make a sample Dataset and Pandas for Transformation
#Used Sqlite3 for Querying

In [48]:
%load_ext sql
import random
from faker import Faker
import pandas as pd

fake = Faker()

# Generation of customers
customers = [{"customer_id": i, "customer_name": fake.name(), "shipping_address": fake.address(), "contact_number": fake.phone_number()} for i in range(1, 11)]

# Generation of products
products = [{"product_id": i, "product_name": fake.word()} for i in range(1, 11)]

# Generation of product variants
product_variants = [{"variant_id": i, "product_id": random.randint(1, 10), "variant_name": fake.word(), "attributes": fake.word()} for i in range(1, 6)]

# Generation of prices
prices = []
for variant in product_variants:
    start_date = fake.date_between(start_date="-2y", end_date="today")
    end_date = fake.date_between(start_date=start_date, end_date="today")
    prices.append({"price_id": len(prices) + 1, "product_variant_id": variant["variant_id"], "price": random.uniform(10, 100), "start_date": start_date, "end_date": end_date})

# Generation of orders
orders = []
for _ in range(100):
    order_date = fake.date_between(start_date="-2y", end_date="today")
    orders.append({"order_id": len(orders) + 1, "customer_id": random.randint(1, 10), "product_variant_id": random.randint(1, 5), "order_date": order_date, "quantity": random.randint(1, 5), "total_price": random.uniform(20, 200)})

# Generate product categories
product_categories = [{"product_id": i, "category_id": random.randint(1, 3)} for i in range(1, 11)] 
# Conversion to DataFrames
customers_df = pd.DataFrame(customers)
products_df = pd.DataFrame(products)
product_variants_df = pd.DataFrame(product_variants)
prices_df = pd.DataFrame(prices)
orders_df = pd.DataFrame(orders)
product_categories_df = pd.DataFrame(product_categories) 

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [50]:
import sqlite3

conn = sqlite3.connect("ecommerce.db")
cursor = conn.cursor()

# Inserting data into Customer_Dim
customers_df.to_sql("Customer_Dim", conn, if_exists="replace", index=False)

# Inserting data into Product_Dim
products_df.to_sql("Product_Dim", conn, if_exists="replace", index=False)

# Inserting data into Product_Variant_Dim
product_variants_df.to_sql("Product_Variant_Dim", conn, if_exists="replace", index=False)

# Inserting data into Price_Dim
prices_df.to_sql("Price_Dim", conn, if_exists="replace", index=False)

# Inserting data into Order_Fact
orders_df.to_sql("Order_Fact", conn, if_exists="replace", index=False)

conn.close()



In [35]:
%sql sqlite:///ecommerce.db

In [51]:
%%sql
#Query 1:Retrieve the top 5 customers who have made the highest average order amounts in the last 6 months.The average order amount should be calculated for each customer, and the result should be sorted in descending order.
SELECT
    c.customer_id,
    c.customer_name,
    AVG(o.total_price) AS average_order_amount
FROM
    Customer_Dim c
JOIN
    Order_Fact o ON c.customer_id = o.customer_id
WHERE
    o.order_date >= DATE('now', '-6 months')
GROUP BY
    c.customer_id, c.customer_name
ORDER BY
    average_order_amount DESC
LIMIT 5;


 * sqlite:///ecommerce.db
Done.


customer_id,customer_name,average_order_amount
1,Alice Wood,151.40817139192052
7,Adrienne Underwood,140.4409302075685
3,Robin Flores,135.49090756765702
2,Megan Howard,119.5897471061988
9,Benjamin Watts,116.92466985813819


In [52]:
#Query2:Retrieve the list of customer whose order value is lower this year as compared to previous year
%%sql
SELECT DISTINCT
    c.customer_id,
    c.customer_name
FROM
    Customer_Dim c
JOIN
    Order_Fact o ON c.customer_id = o.customer_id
WHERE
    strftime('%Y', o.order_date) = strftime('%Y', 'now')
    AND o.total_price < (
        SELECT SUM(total_price)
        FROM Order_Fact
        WHERE customer_id = c.customer_id AND strftime('%Y', order_date) = strftime('%Y', 'now', '-1 year')
    );


 * sqlite:///ecommerce.db
Done.


customer_id,customer_name
6,Stephanie Bowers
7,Adrienne Underwood
10,Amanda Burke
5,Stephanie Woods
3,Robin Flores
9,Benjamin Watts
2,Megan Howard
8,Mark Davis
1,Alice Wood
4,Christina Miller


In [55]:
#Query3(a):Create a table showing cumulative purchase by a particular customer. 
%%sql
SELECT
    o.customer_id,
    c.customer_name,
    SUM(o.total_price) AS cumulative_purchase
FROM
    Order_Fact o
JOIN
    Customer_Dim c ON o.customer_id = c.customer_id
GROUP BY
    o.customer_id, c.customer_name
ORDER BY
    cumulative_purchase DESC;


 * sqlite:///ecommerce.db
Done.


customer_id,customer_name,cumulative_purchase
2,Megan Howard,1573.4923141021209
1,Alice Wood,1514.4936728264558
3,Robin Flores,1446.8489458697022
6,Stephanie Bowers,1424.2105101246393
4,Christina Miller,1329.7474747028307
7,Adrienne Underwood,1099.2307935186855
10,Amanda Burke,991.3416951148538
9,Benjamin Watts,854.1822038123443
8,Mark Davis,664.1018627156567
5,Stephanie Woods,658.977179758091


In [57]:
#Query3(b):Show the breakup of cumulative purchases by product category
%%sql
SELECT
    c.customer_id,
    c.customer_name,
    p.product_name,
    SUM(o.total_price) AS cumulative_purchase
FROM
    Order_Fact o
JOIN
    Customer_Dim c ON o.customer_id = c.customer_id
JOIN
    Product_Variant_Dim pv ON o.product_variant_id = pv.variant_id
JOIN
    Product_Dim p ON pv.product_id = p.product_id
GROUP BY
    c.customer_id, c.customer_name, p.product_name
ORDER BY
    c.customer_id, cumulative_purchase DESC;


 * sqlite:///ecommerce.db
Done.


customer_id,customer_name,product_name,cumulative_purchase
1,Alice Wood,decision,847.0158630899434
1,Alice Wood,large,336.6675798447559
1,Alice Wood,development,330.81022989175653
2,Megan Howard,large,650.7572262171009
2,Megan Howard,decision,384.00433145675163
2,Megan Howard,seat,342.3355655796042
2,Megan Howard,development,196.39519084866404
3,Robin Flores,decision,748.211550760499
3,Robin Flores,seat,445.18372932682416
3,Robin Flores,large,161.83891498304064


In [59]:
#Query4:Retrieve the list of top 5 selling products. Further bifurcate the sales by product variants
%%sql
SELECT
    p.product_name,
    pv.variant_name,
    SUM(o.quantity) AS total_quantity_sold
FROM
    Order_Fact o
JOIN
    Product_Variant_Dim pv ON o.product_variant_id = pv.variant_id
JOIN
    Product_Dim p ON pv.product_id = p.product_id
GROUP BY
    p.product_name, pv.variant_name
ORDER BY
    total_quantity_sold DESC
LIMIT 5;


 * sqlite:///ecommerce.db
Done.


product_name,variant_name,total_quantity_sold
large,loss,86
decision,develop,70
seat,second,60
development,continue,53
decision,political,43
